<a href="https://colab.research.google.com/github/logmosier/ai_project/blob/collab/mario_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install dependancies

Rendering Dependancies



In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Main Dependancies

In [ ]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)
!pip install gym
#!pip install git+https://github.com/sajjadzamani/gym-super-mario-bros.git
!pip install gym-super-mario-bros
!pip install stable_baselines==2.10.0

# Setup Video Rendering


In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('./video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', video_callable=lambda episode_id: True, force=True)
  return env

In [ ]:
from tqdm.notebook import tqdm
from stable_baselines.common.callbacks import BaseCallback

class TqdmCallback(BaseCallback):
  def __init__(self, verbose=0):
        super(TqdmCallback, self).__init__(verbose)
        self.pbar = None

  def _on_step(self):
    if self.pbar is None:
      self.pbar = tqdm(total=self.locals['total_timesteps'])   
    self.pbar.update(1)
      
    if self.locals['total_timesteps'] == self.num_timesteps:
        self.pbar.close()
        self.pbar = None

    return True

# Build Model

In [ ]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY
from stable_baselines.common.policies import MlpPolicy, CnnPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import PPO2
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.callbacks import CallbackList, CheckpointCallback, EvalCallback

import gym


MODEL_NAME = 'mario'
ENV_ID = 'SuperMarioBros-1-1-v2'
LEARN_RATE = 0.001
TIME_STEPS=10000
POLICY = MlpPolicy
EPOCH = 6

# multiprocess environment
env = gym_super_mario_bros.make(ENV_ID)
env = JoypadSpace(env, RIGHT_ONLY)
#env = wrap_env(env)

# Save a checkpoint every 1000 steps
# callback = CheckpointCallback(save_freq=1000000, save_path='./logs_joypad_vec/',
                                      #  name_prefix='rl_model')
callback = TqdmCallback()

model = PPO2(POLICY, env, learning_rate=LEARN_RATE, noptepochs=EPOCH)
model.learn(total_timesteps=TIME_STEPS, callback=callback)
model.save(MODEL_NAME)


In [ ]:
from stable_baselines.common.vec_env import VecVideoRecorder, DummyVecEnv
video_folder = './video'
video_length = 10000

env.reset()
env = DummyVecEnv([lambda: env])
obs = env.reset()

# Record the video starting at the first step
env = VecVideoRecorder(env, video_folder,
                       record_video_trigger=lambda x: x == 0, video_length=video_length,
                       name_prefix="random-agent-{}".format(ENV_ID))

env.reset()
for _ in range(video_length + 1):
  action, _states = model.predict(obs)
  obs, _, _, _ = env.step(action)
# Save the video
env.close()
show_video()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')